In [2]:
from pyspark import SparkContext,SparkConf

In [3]:
sc=SparkContext()

In [4]:
data_file="/data/kdd/kddcup.data_10_percent"
raw_data=sc.textFile(data_file)

In [5]:
print "Train data size is {}".format(raw_data.count())

Train data size is 494021


In [6]:
test_data_file="/data/kdd/corrected"
test_raw_data=sc.textFile(test_data_file)
print "Test data size is {}".format(test_raw_data.count())

Test data size is 311029


In [7]:
from pyspark.mllib.regression import LabeledPoint
from numpy import array

csv_data = raw_data.map(lambda x: x.split(","))
test_csv_data = test_raw_data.map(lambda x: x.split(","))

protocols = csv_data.map(lambda x: x[1]).distinct().collect()
services = csv_data.map(lambda x: x[2]).distinct().collect()
flags = csv_data.map(lambda x: x[3]).distinct().collect()

In [8]:
def create_labeled_point(line_split):
    # leave_out = [41]
    clean_line_split = line_split[0:41]
    
    # convert protocol to numeric categorical variable
    try: 
        clean_line_split[1] = protocols.index(clean_line_split[1])
    except:
        clean_line_split[1] = len(protocols)
        
    # convert service to numeric categorical variable
    try:
        clean_line_split[2] = services.index(clean_line_split[2])
    except:
        clean_line_split[2] = len(services)
    
    # convert flag to numeric categorical variable
    try:
        clean_line_split[3] = flags.index(clean_line_split[3])
    except:
        clean_line_split[3] = len(flags)
    
    # convert label to binary label
    attack = 1.0
    if line_split[41]=='normal.':
        attack = 0.0
        
    return LabeledPoint(attack, array([float(x) for x in clean_line_split]))

training_data = csv_data.map(create_labeled_point)
test_data = test_csv_data.map(create_labeled_point)

In [9]:
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
from time import time

# Build the model
t0 = time()
model = NaiveBayes.train(training_data, 1.0)
tt = time() - t0

print "Classifier trained in {} seconds".format(round(tt,3))

Classifier trained in 11.567 seconds


In [10]:
predictions = model.predict(test_data.map(lambda p: p.features))
labels_and_preds = test_data.map(lambda p: p.label).zip(predictions)

In [11]:
t0 = time()
test_accuracy = labels_and_preds.filter(lambda (v, p): v == p).count() / float(test_data.count())
tt = time() - t0

print "Prediction made in {} seconds. Test accuracy is {}".format(round(tt,3), round(test_accuracy,4))

Prediction made in 12.037 seconds. Test accuracy is 0.918
